In [1]:
import torch
import torch.nn as nn

Neural networks can be constructed using the `torch.nn` package.

A typical training procedure for a NN is as follows:

+ Define the neural network that has some learnable parameters (or weights)
+ Iterate over a dataset of inputs
+ Process input through the network
+ Compute the loss (how far is the output from being correct)
+ Propagate gradients back into the network's parameters
+ Update the weights of the network, typically using a simple update rule
`weight = weight - learning_rate * gradient`

### Define the network

In [11]:
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        #an affine operation y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    
    def forward(self, x):
        # max pooling over a (2, 2) kernel
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        # if the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_feature(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_feature(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [12]:
params = list(net.parameters())
print(len(params))
print(params[0].size())
print(params[1].size())
# print(params)

10
torch.Size([6, 1, 5, 5])
torch.Size([6])


Let try a random 32x32 input.

Note: expected input size to this net (LeNet) is 32x32. To use this net on MNIST dataset, pls resize the imges from the dataset to 32x32

In [13]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0501,  0.1653,  0.0008,  0.0085, -0.0491, -0.0001,  0.0700, -0.0048,
         -0.0328, -0.1096]], grad_fn=<ThAddmmBackward>)


In [14]:
net.zero_grad()
out.backward(torch.randn(1, 10))